# Project 2 : :)
## Authors:

### Warnings

* Ikke endre uten at de andre vet det
* **Alltid** gå på Kernel -> Restart & Clear Output
* Ikke skriv utenfor cellene og sånn

## Implementation of set 1 - points in 2D

In [ ]:
## IMPORTS
import numpy as np
import matplotlib.pyplot as plt
import loader
import plotting
from spirals import get_data_spiral_2d

## GLOBAL CONSTS AND DEFINITIONS
sigma = lambda x : np.tanh(x)         # Activation function
eta = lambda x : 0.5*(1+np.tanh(x/2)) # Scalar function     
sech = lambda x : 1/np.cosh(x)
eta_ddx = lambda x :  0.25*sech(x/2)*sech(x/2) # Derivative of eta
sigma_ddx = lambda x: sech(x)*sech(x) # Derivative of sigma
h = 0.1                               # Step length          
#_____________________________________________________________

One = np.full((I,2),1)
print(I)

In [ ]:
# Algorithm
K = 4 # Amount of layers, arbitrary number
d = 3 # Ehh
tau = 0.07 # learning parameter [0.01,0.1]
Y0 = readData() # To be implemented
Wk, bk = np.random.randn(K,d,d), np.random.randn(K,d,1)
converged = False
while not converged:
    for k in range(1,K):
        #Calculate Y[k] and save to memory
    getGradients()
    AdamAlg()
    converged = True #Only one iteration when debugging

In [ ]:
from spirals import get_data_spiral_2d
import numpy as np
Y0, C = get_data_spiral_2d() # Default n=200
I = Y0.shape[1]
W = np.array([[1,1], [1,1]])
K=4
#b = np.array([[1],[1]])
b = np.ones((K,2,1))
#b[k-1,:,:]
#b = np.array([[1 for i in range(I)], [1 for i in range(I)]]) # [1, 1, 1, ... i times], [1,1,1, .. i times], in order to add bk
K = 4 #debug
mu = 1 #?
w = 1 #?
h = 0.1
def Y(k, W, b):
    assert k>=0
    if k==0:
        return Y0
    else:
        return Y(k-1, W, b)+h*sigma(W@Y(k-1, W, b)+b)
lastY = Y(K, W, b)[K-1]
print(lastY.shape)


Z = lambda w,mu : eta(np.add(np.transpose(lastY)*w,np.ones((200,2)))) # NB SPØR STUDASS
print(":)))")
J = 1/2*np.linalg.norm(Z(w,mu)-C)**2 # Cost function
print(J)

In [ ]:


## Adam descent algorithn
def AdamAlg():
    beta1 = 0.9
    beta2 = 0.999
    alpha = 0.01
    epsilon = 1e-8
    g = getGradients()
    if counter==1:
        lastM = 0
        lastV = 0
    m = beta1*lastM+(1-beta1)*g
    v = beta2*lastV+(1-beta2)*(g*g)
    mhat = m/(1-beta1**(counter)) # NB må altså 
    vhat = v/(1-beta2**(counter))
    U -= alpha*mhat/(np.sqrt(vhat)+epsilon)
    lastM = m
    lastV = v

    
def getGradients():
    P = np.full(Y0.shape,0) # init matrix
    P[K] = w*np.transpose((Z(w,mu)-C)*eta_ddx(np.transpose(Y(K,W,b))*w+mu*One))
    dJdMU = eta_ddx(np.transpose(np.transpose(Y(K,W,b))*w+mu*One)*(Z(w,mu)-C))
    dJdw = Y(K,W,b)*((Z(w,mu)-C)*eta_ddx(np.transpose(Y(K,W,b))*w+mu))
    for k in range(K,0,-1):
        P[k-1] = P[k]+h*np.transpose(W[k-1])*(sigma_ddx(W[k-1,:,:]*Y(k-1,W,b)+b[k-1,:,:])*P[k])
    for k in range(0,K-1):
        dJdWk[k] = h(P[k+1]*sigma_ddx(W[k,:,:]*Y(k,W,b)+b[k,:,:]))@np.transpose(Y(k,W,b))
        dJdBk[k] = h(P[k+1]*sigma_ddx(W[k,:,:]*Y(k,W,b)+b[k,:,:]))@One
    return dJdWk, dJdBk, dJdw, dJdMU


# First for-loop in getGradients, is the range correct?
# Parameters for Adam alg?
# What is Y[k]? svart
# How to readData()? svart
# How to check if converged? And when? svart
# Hvordan få posisjon til farge? svart
# Må vi ha forskjellgie b_k? Hva menes med b_{k-1}?
# Hva gjør egentlig Adam Descent Alg.? (Hva er U?)
# Random startverdier for vekter og bias: på hvilket intervall??
# Hvor mange verdier for Bk og Wk må vi lage?
# Er Wk=2x2 og Bk=2x1?

In [ ]:
#### OLD BUT GOLD ;)


# Old adam algorithm
    g = [0 for i in range(K)]
    m = [0 for i in range(K)]
    v = [0 for i in range(K)]
    mhat = [0 for i in range(K)]
    vhat = [0 for i in range(K)]
    v[0] = 0
    m[0]
    for j in range(999):
        g[j] = uGrad(J(U[j]))
        m[j] = beta1*m[j-1]+(1-beta1)*g[j]
        v[j] = beta2*v[j-1]+(1-beta2)(g[j]*g[j])
        mhat[j] = m[j]/(1-beta1**(j+1))
        vhat[j] = v[j]/(1-beta2**(j+1))
        U[j+1] -= alpha*mhat[j]/(np.sqrt(vhat[j])+e)

In [ ]:
matrix1 = np.array([[1,2,3], [2,3,4]])
matrix2 = np.array([[1,4,8], [0,2,6], [1,6,2]])
print(matrix1+matrix2)